<a href="https://colab.research.google.com/github/stevalang/Data-Science-Lessons/blob/main/Copy_of_artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Part 1 - Data Preprocessing

In [31]:
tf.__version__

'2.4.1'

### Importing the dataset

In [32]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [33]:
y

array([1, 0, 1, ..., 1, 1, 0])

### Encoding categorical data

In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

Label Encoding the "Gender" column

One Hot Encoding the "Geography" column

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [36]:
X.shape

(10000, 12)

### Splitting the dataset into the Training set and Test set

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling

In [38]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [39]:
X_train

array([[-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
        -1.03227043,  0.32472465]])

## Part 2 - Building the ANN

In [40]:
ann = tf.keras.models.Sequential()

### Initializing the ANN

### Adding the input layer and the first hidden layer

In [43]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [44]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [45]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [46]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN on the Training set

In [47]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5623 - accuracy: 0.7563
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4788 - accuracy: 0.7954
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4490 - accuracy: 0.7937
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4247 - accuracy: 0.8139
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4197 - accuracy: 0.8188
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4087 - accuracy: 0.8212
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3989 - accuracy: 0.8285
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4005 - accuracy: 0.8289
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3925 - accuracy: 0.8284
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3972 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [51]:
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5

array([[False]])

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [53]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1)

array([[0, 0],
       [0, 1],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

### Making the Confusion Matrix

In [54]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1521   74]
 [ 201  204]]


0.8625